In [23]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import numpy as np

In [24]:
torch.cuda.set_device(0)
device = torch.device("cuda")

In [25]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

In [30]:
trainset = datasets.ImageFolder("train/", transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.DatasetFolder("test_all/", transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

TypeError: DatasetFolder.__init__() missing 1 required positional argument: 'loader'

In [29]:
trainset.classes

['acoustic',
 'antenna',
 'bacteria',
 'battery',
 'bean',
 'beetle',
 'bicycle',
 'birch',
 'bird',
 'bomb',
 'bread',
 'bridge',
 'camera',
 'carbon',
 'cat',
 'corn',
 'crab',
 'crocodilian',
 'echinoderm',
 'egg',
 'elephant',
 'fish',
 'flower',
 'frog',
 'fungus',
 'gauge',
 'hammer',
 'icecream',
 'kangaroo',
 'memorial',
 'monkey',
 'motor',
 'nest',
 'palm',
 'pizza',
 'pot',
 'printer',
 'saw',
 'snake',
 'spice',
 'spider',
 'spoon',
 'squash',
 'swine',
 'tea',
 'tomato',
 'towel',
 'truck',
 'turtle',
 'worm']